### Project Managment
$$\begin{array}{rll}
& T = \text{time the project is complete}\\
& s_{A} = \text{start time for activity A}\\
& s_{B} = \text{start time for activity B}\\
& s_{C} = \text{start time for activity C}\\
& s_{D} = \text{start time for activity D}\\
& s_{E} = \text{start time for activity E}\\
& s_{F} = \text{start time for activity F}\\
& s_{G} = \text{start time for activity G}\\
\end{array}
$$

$$\begin{array}{rll}
 \text{min} & T\\
 \text{s.t.} & s_{B} \ge s_{A} + 21 \text{ (Activity B cannot start until the completion time of A)}\\
 & s_{C} \ge s_{A} + 21 \text{ (Activity C cannot start until the completion time of A)}\\
 & s_{D} \ge s_{B} + 5 \text{ (Activity D cannot start until the completion time of B)}\\
 & s_{E} \ge s_{C} + 7.5 \text{ (Activity E cannot start until the completion time of C)}\\
 & s_{E} \ge s_{D} + 2 \text{ (Activity E cannot start until the completion time of D)}\\
 & s_{F} \ge s_{C} + 7.5 \text{ (Activity F cannot start until the completion time of C)}\\
 & s_{F} \ge s_{D} + 2 \text{ (Activity F cannot start until the completion time of D)}\\
 & s_{G} \ge s_{E} + 7.5 \text{ (Activity G cannot start until the completion time of E)}\\
 & s_{G} \ge s_{F} + 9 \text{ (Activity G cannot start until the completion time of F)}\\
 & T \ge s_{G} + 2 \text{ (The project cannot finish until the completion time of G)}\\
 & \text{non-negativity constraint}
\end{array}
$$

### Set up Gurobi environment

In [1]:
from gurobipy import *
m = Model()

### Set up activities, predecessors, and durations

In [2]:
activities = ["A", "B", "C", "D", "E", "F", "G", "T"]

In [3]:
duration = {"A":21, "B":5, "C":7.5, "D":2, "E":7.5, "F":9, "G":2}

predecessors = {"A":{},
                "B":{"A"},
                "C":{"A"},
                "D":{"B"},
                "E":{"C","D"},
                "F":{"C","D"},
                "G":{"E","F"},
                "T":{"G"}
               }

In [4]:
time_contribution = {"A":0, "B":0, "C":0, "D":0, "E":0, "F":0, "G":0, "T":1}

### Add variables and constraints into the Gurobi model

In [5]:
activities_time = m.addVars(activities,  name="activities") 

In [6]:
for a in activities:
    activities_time[a].vtype = GRB.CONTINUOUS
    activities_time[a].lb = 0

### Constraints for predecessors

In [7]:
for a in activities:
    for p in predecessors[a]:
        m.addConstr(activities_time[a] >= activities_time[p] + duration[p])

### Constraints to make sure Y is the finish time for the last activity

In [8]:
m.update()

In [9]:
obj = activities_time["T"]

In [10]:
m.setObjective(obj, GRB.MINIMIZE)

### Solve the LP, and look for the optimal objective function value

In [11]:
m.optimize()

Optimize a model with 10 rows, 8 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 10 rows and 8 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.06 seconds
Optimal objective  3.950000000e+01


### Optimal starting time for each activity

In [12]:
m.printAttr('X')


    Variable            X 
-------------------------
activities[B]           21 
activities[C]           21 
activities[D]           26 
activities[E]         28.5 
activities[F]         28.5 
activities[G]         37.5 
activities[T]         39.5 
